# Freqtrade strategy snippets for better strategies

## De volledige strategie uitgeschreven:

Timeframe: 2hr of hoger (afhankelijk van backtest resultaten)

### Instapsignaal

* Closeprice > Keltnerchannel upper band &
* Closeproce > highest close van de afgelopen 20 candles &
* MACD lijn > Signallijn &
* Slow stochastic > 50

### Take profit

* Take profit prijs is de highest low koers van de laatste retracement (10 candles daarvoor). Met andere woorden, onder normale omstandigheden stijgt de prijs met kleine retracements (AB-BC-CD patroon). Als de koers echter onder de laagste retracement sluit (punt C sluit onder punt A), dan is volgens deze strategie de verwachting dat er ook lower highs zullen komen. Met andere woorden een reversal.

### Initial stoploss

* 4% initial stol-loss als de trade ogenblikkelijk tegen ons keert of als we met de strategie te hoog zijn ingestapt (ivm voldoen aan condities).

### Trailing stoploss



## Basic dataframe configuration and prerequisites

In [2]:
# Import necessary libraries
import pandas as pd
from freqtrade.strategy.interface import IStrategy
from pandas import DataFrame
import talib.abstract as ta
import freqtrade.vendor.qtpylib.indicators as qtpylib
from scipy.spatial.distance import cosine
import numpy as np

In [23]:
# Import data for analysis and name columns
df = pd.read_json("BTC_USDT-2h.json")
df.columns=['date','open','high','low','close','volume']
df['date']=(pd.to_datetime(df['date'],unit='ms'))
df.head(5)

,date,open,high,low,close,volume
0,2017-08-17 04:00:00,4261.48,4328.69,4261.32,4315.32,70.415925
1,2017-08-17 06:00:00,4330.29,4349.99,4287.41,4349.99,11.672940
2,2017-08-17 08:00:00,4333.32,4445.78,4333.32,4444.00,11.736430
3,2017-08-17 10:00:00,4441.10,4485.39,4399.81,4427.30,51.883452
4,2017-08-17 12:00:00,4436.06,4459.00,4411.00,4459.00,54.344834


### Simple moving average and RSI

In [49]:
df['sma'] = ta.SMA(df, timeperiod=21)
df['rsi'] = ta.RSI(df, timeperiod=7)
print(df[['date','close', 'low','sma','rsi']].tail(25))

                     date     close       low           sma        rsi
15889 2021-04-06 06:00:00  58576.59  58349.00  58228.831429  51.095218
15890 2021-04-06 08:00:00  58685.53  58300.00  58265.375238  53.840781
15891 2021-04-06 10:00:00  58541.34  58325.08  58282.628095  49.545616
15892 2021-04-06 12:00:00  58414.29  58342.00  58304.553333  45.790445
15893 2021-04-06 14:00:00  57818.53  57413.02  58290.355238  32.369044
15894 2021-04-06 16:00:00  58247.17  57722.23  58292.505714  45.722926
15895 2021-04-06 18:00:00  58225.00  57996.00  58313.320952  45.184567
15896 2021-04-06 20:00:00  57860.63  57555.00  58332.059524  36.862263
15897 2021-04-06 22:00:00  57991.15  57483.71  58384.140476  41.374783
15898 2021-04-07 00:00:00  57510.84  57161.86  58396.561429  31.660012
15899 2021-04-07 02:00:00  57922.75  57475.86  58414.828095  44.660498
15900 2021-04-07 04:00:00  58285.99  57268.00  58440.048571  53.718462
15901 2021-04-07 06:00:00  57850.57  57420.00  58427.549048  43.712432
15902 

### MACD

In [25]:
macd = ta.MACD(
    df,
    fastperiod=12,
    fastmatype=0,
    slowperiod=26,
    slowmatype=0,
    signalperiod=9,
    signalmatype=0,
    )
df["macd"] = macd["macd"]
df["macdsignal"] = macd["macdsignal"]
df["macdhist"] = macd["macdhist"]

In [26]:
print(df[['date','close', 'sma','rsi','macd','macdsignal','macdhist']].tail(5))

                     date     close           sma        rsi        macd  \
15909 2021-04-07 22:00:00  55953.45  57489.958095  27.455918 -582.374709   
15910 2021-04-08 00:00:00  56504.14  57391.270000  43.526028 -560.012995   
15911 2021-04-08 02:00:00  56540.62  57289.131429  44.476605 -533.201133   
15912 2021-04-08 04:00:00  56871.90  57209.634286  52.879617 -479.691427   
15913 2021-04-08 06:00:00  57025.71  57143.511429  56.449760 -420.031556   

       macdsignal    macdhist  
15909 -397.666064 -184.708645  
15910 -430.135450 -129.877544  
15911 -450.748587  -82.452546  
15912 -456.537155  -23.154272  
15913 -449.236035   29.204479  


### Slow Stochastic

In [27]:
stoch = ta.STOCH(
    df,
    fastk_period=14,
    slowk_period=3,
    slowk_matype=0,
    slowd_period=3,
    slowd_matype=0,
    )
df["slowd"] = stoch["slowd"]
df["slowk"] = stoch["slowk"]

print(df[['date','close', 'slowd','slowk']].tail(5))

                     date     close      slowd      slowk
15909 2021-04-07 22:00:00  55953.45  19.884135  19.840876
15910 2021-04-08 00:00:00  56504.14  20.987715  23.510580
15911 2021-04-08 02:00:00  56540.62  23.456736  27.018751
15912 2021-04-08 04:00:00  56871.90  29.056463  36.640059
15913 2021-04-08 06:00:00  57025.71  35.254208  42.103813


### Keltner Channel

In [28]:
keltner = qtpylib.keltner_channel(df, window=21, atrs=1)
df["kc_upperband"] = keltner["upper"]
df["kc_lowerband"] = keltner["lower"]
df["kc_middleband"] = keltner["mid"]
df["kc_percent"] = (df["close"] - df["kc_lowerband"]) / (
    df["kc_upperband"] - df["kc_lowerband"]
)
df["kc_width"] = (
    df["kc_upperband"] - df["kc_lowerband"]
) / df["kc_middleband"]


In [29]:
print(df[['date','close', 'kc_upperband','kc_middleband','kc_lowerband','kc_percent','kc_width']].tail(5))

                     date     close  kc_upperband  kc_middleband  \
15909 2021-04-07 22:00:00  55953.45  58325.243492   57487.761111   
15910 2021-04-08 00:00:00  56504.14  58232.378571   57376.801429   
15911 2021-04-08 02:00:00  56540.62  58116.046032   57272.190794   
15912 2021-04-08 04:00:00  56871.90  58041.367937   57190.901746   
15913 2021-04-08 06:00:00  57025.71  57961.290476   57118.633810   

       kc_lowerband  kc_percent  kc_width  
15909  56650.278730   -0.416026  0.029136  
15910  56521.224286   -0.009984  0.029823  
15911  56428.335556    0.066531  0.029468  
15912  56340.435556    0.312455  0.029741  
15913  56275.977143    0.444863  0.029505  


## Search for values in arrays for strategy

## Highest / Lowest value in array

To determine the bottom / top of a swing. Stop-loss percentage should be derived from it.
My idea is to calculate the difference in percentage because freqtrade only knows percentages when determine initial stop-losses. 

Calculate the difference in percentage between entry price and current lowest low of swing.

An alternative is to calculate difference between the entry price and ATR stop loss in percentages.

In [44]:
# Eerst alles tonen voor controle
df_lowest = df.iloc[-3:]
print(df_lowest)

                     date      open      high       low     close  \
15911 2021-04-08 02:00:00  56504.13  56654.14  56200.30  56540.62   
15912 2021-04-08 04:00:00  56540.62  57096.46  56529.78  56871.90   
15913 2021-04-08 06:00:00  56871.89  57333.85  56841.85  57025.71   

            volume           sma        rsi        macd  macdsignal  \
15911  2917.142060  57289.131429  44.476605 -533.201133 -450.748587   
15912  3917.144907  57209.634286  52.879617 -479.691427 -456.537155   
15913  3781.964426  57143.511429  56.449760 -420.031556 -449.236035   

        macdhist      slowd      slowk  kc_upperband  kc_lowerband  \
15911 -82.452546  23.456736  27.018751  58116.046032  56428.335556   
15912 -23.154272  29.056463  36.640059  58041.367937  56340.435556   
15913  29.204479  35.254208  42.103813  57961.290476  56275.977143   

       kc_middleband  kc_percent  kc_width  
15911   57272.190794    0.066531  0.029468  
15912   57190.901746    0.312455  0.029741  
15913   57118.633810  

In [42]:
# Nu alleen van kolom 'close' laatste 3 waardes tonen
df_lowest = df['close'].iloc[-3:]
print(df_lowest)

15911    56540.62
15912    56871.90
15913    57025.71
Name: close, dtype: float64


In [50]:
# Nu alleen van kolom 'close' laatste 25 waardes tonen
df_lowest = df['low'].iloc[-25:]
print(df_lowest)

15889    58349.00
15890    58300.00
15891    58325.08
15892    58342.00
15893    57413.02
15894    57722.23
15895    57996.00
15896    57555.00
15897    57483.71
15898    57161.86
15899    57475.86
15900    57268.00
15901    57420.00
15902    57000.00
15903    55700.00
15904    55612.00
15905    56275.00
15906    55972.33
15907    55473.00
15908    55717.15
15909    55865.11
15910    55700.00
15911    56200.30
15912    56529.78
15913    56841.85
Name: low, dtype: float64


In [52]:
# Bepaal de laagste waarde van deze array
print(df_lowest.min())

55473.0


In [79]:
# Dit klopt :-)
# Nu van een andere willekeurige array in deze tabel (df.iloc[row_index])
# df_sub = df['low'].iloc[15880:15897]
# df_sub = df['low'].iloc[-20:-1]
df_low_array = df['low'].iloc[-225:-200]
print(df_low_array)

15689    58755.30
15690    58901.66
15691    59104.28
15692    58131.55
15693    57966.85
15694    57811.41
15695    57363.07
15696    56717.81
15697    56668.00
15698    55450.11
15699    55605.94
15700    55870.23
15701    57000.05
15702    56750.00
15703    57186.82
15704    57474.46
15705    57128.89
15706    56289.00
15707    56418.55
15708    57315.22
15709    57513.41
15710    57024.13
15711    57282.00
15712    56521.01
15713    56778.52
Name: low, dtype: float64


In [88]:
# Determine lowest value within array
df_lowest = df_low_array.min()
df_highest = df_low_array.max()
print('Lowest value is: '+ str(df_lowest) + ' - and highest value is: ' + str(df_highest))

Lowest value is: 55450.11 - and highest value is: 59104.28


Right again. We can use this formula.

Determine for each row the lowest value from the last 25 candles.

To make use of changing values, the .rolling(window) subset is used to determing the lowest value within the rolling window. See: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

In [158]:
# Determine the lowest value from the last 25 candles in a rolling window.
df['last_lowest'] = df['low'].rolling(10).min()
df['last_highest'] = df['high'].rolling(10).max().shift(1)
# Or with closed:
# dataframe['last_highest'] = dataframe['close'].rolling(20).max().shift(1)
print(df[['date','low','last_lowest','high','last_highest']].tail(20))

                     date       low  last_lowest      high  last_highest
15894 2021-04-06 16:00:00  57722.23     57413.02  58271.66      59495.24
15895 2021-04-06 18:00:00  57996.00     57413.02  58282.12      59495.24
15896 2021-04-06 20:00:00  57555.00     57413.02  58311.91      59495.24
15897 2021-04-06 22:00:00  57483.71     57413.02  58163.00      59031.66
15898 2021-04-07 00:00:00  57161.86     57161.86  58200.01      59031.66
15899 2021-04-07 02:00:00  57475.86     57161.86  58124.75      59000.00
15900 2021-04-07 04:00:00  57268.00     57161.86  58317.06      59000.00
15901 2021-04-07 06:00:00  57420.00     57161.86  58655.00      58998.00
15902 2021-04-07 08:00:00  57000.00     57000.00  57938.77      58998.00
15903 2021-04-07 10:00:00  55700.00     55700.00  57610.60      58655.00
15904 2021-04-07 12:00:00  55612.00     55612.00  56816.72      58655.00
15905 2021-04-07 14:00:00  56275.00     55612.00  56937.15      58655.00
15906 2021-04-07 16:00:00  55972.33     55612.00  5

In [143]:
# Determine the percentage difference between the closing price and the last lowest price.
# df['psl'] = 1-(df['close'] / df['last_lowest'])
df['psl'] = 1-(df['close'] / (df['last_lowest']-10))
df['sl'] = df['psl'].round(decimals=3)
print(df[['date','close','low','last_lowest','sl']].tail(20))

                     date     close       low  last_lowest     sl
15894 2021-04-06 16:00:00  58247.17  57722.23     57413.02 -0.015
15895 2021-04-06 18:00:00  58225.00  57996.00     57413.02 -0.014
15896 2021-04-06 20:00:00  57860.63  57555.00     57413.02 -0.008
15897 2021-04-06 22:00:00  57991.15  57483.71     57413.02 -0.010
15898 2021-04-07 00:00:00  57510.84  57161.86     57161.86 -0.006
15899 2021-04-07 02:00:00  57922.75  57475.86     57161.86 -0.013
15900 2021-04-07 04:00:00  58285.99  57268.00     57161.86 -0.020
15901 2021-04-07 06:00:00  57850.57  57420.00     57161.86 -0.012
15902 2021-04-07 08:00:00  57466.61  57000.00     57000.00 -0.008
15903 2021-04-07 10:00:00  56320.47  55700.00     55700.00 -0.011
15904 2021-04-07 12:00:00  56756.66  55612.00     55612.00 -0.021
15905 2021-04-07 14:00:00  56451.45  56275.00     55612.00 -0.015
15906 2021-04-07 16:00:00  56050.54  55972.33     55612.00 -0.008
15907 2021-04-07 18:00:00  56153.83  55473.00     55473.00 -0.012
15908 2021

In [144]:
# Tighter stop loss with ATR usage
atr_factor = 0.9
df['atr'] = ta.ATR(df, timeperiod=14)
# ATR stoploss (tradingview uses open price)
df['atr_sl'] = df['open'] - (df['atr'] * atr_factor)
# Freqtrade stoploss value
df['psl'] = 1-(df['close'] / df['atr_sl'])
df['sl'] = df['psl'].round(decimals=3)
# print(df[['date','open','close','atr','atr_sl','psl','sl']].tail(20))
print(df[['date','open','close','atr_sl','sl']].tail(20))

                     date      open     close        atr_sl     sl
15894 2021-04-06 16:00:00  57818.54  58247.17  57255.360882 -0.017
15895 2021-04-06 18:00:00  58247.17  58225.00  57705.824533 -0.009
15896 2021-04-06 20:00:00  58225.01  57860.63  57673.673567 -0.003
15897 2021-04-06 22:00:00  57860.63  57991.15  57305.006097 -0.012
15898 2021-04-07 00:00:00  57990.03  57510.84  57407.355305 -0.002
15899 2021-04-07 02:00:00  57510.84  57922.75  56928.070569 -0.017
15900 2021-04-07 04:00:00  57922.74  58285.99  57314.157385 -0.017
15901 2021-04-07 06:00:00  58286.00  57850.57  57641.494715 -0.004
15902 2021-04-07 08:00:00  57850.57  57466.61  57191.751307 -0.005
15903 2021-04-07 10:00:00  57466.61  56320.47  56732.025499  0.007
15904 2021-04-07 12:00:00  56320.47  56756.66  55560.909535 -0.022
15905 2021-04-07 14:00:00  56753.13  56451.45  56005.257068 -0.008
15906 2021-04-07 16:00:00  56451.46  56050.54  55700.584920 -0.006
15907 2021-04-07 18:00:00  56054.22  56153.83  55293.785997 -0

Er is een manier om bovenstaande informatie uit het dataframe te halen en vervolgens als dictionary weer in te voeren in een andere functie als custom data:

Zie daarvoor: https://www.freqtrade.io/en/stable/strategy-advanced/#storing-custom-information-using-datetimeindex-from-dataframe

```
import talib.abstract as ta
class AwesomeStrategy(IStrategy):
    # Create custom dictionary
    custom_info = {}

    def populate_indicators(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        # using "ATR" here as example
        dataframe['atr'] = ta.ATR(dataframe)
        if self.dp.runmode.value in ('backtest', 'hyperopt'):
          # add indicator mapped to correct DatetimeIndex to custom_info
          self.custom_info[metadata['pair']] = dataframe[['date', 'atr']].copy().set_index('date')
        return dataframe

```

Daarna kun je deze custom informatie in de custom stop loss strategie plaatsen door:

```
from freqtrade.persistence import Trade
from freqtrade.state import RunMode

class AwesomeStrategy(IStrategy):

    # ... populate_* methods

    use_custom_stoploss = True

    def custom_stoploss(self, pair: str, trade: 'Trade', current_time: datetime,
                        current_rate: float, current_profit: float, **kwargs) -> float:

        result = 1
        if self.custom_info and pair in self.custom_info and trade:
            # using current_time directly (like below) will only work in backtesting.
            # so check "runmode" to make sure that it's only used in backtesting/hyperopt
            if self.dp and self.dp.runmode.value in ('backtest', 'hyperopt'):
              relative_sl = self.custom_info[pair].loc[current_time]['atr']
            # in live / dry-run, it'll be really the current time
            else:
              # but we can just use the last entry from an already analyzed dataframe instead
              dataframe, last_updated = self.dp.get_analyzed_dataframe(pair=pair,
                                                                       timeframe=self.timeframe)
              # WARNING
              # only use .iat[-1] in live mode, not in backtesting/hyperopt
              # otherwise you will look into the future
              # see: https://www.freqtrade.io/en/latest/strategy-customization/#common-mistakes-when-developing-strategies
              relative_sl = dataframe['atr'].iat[-1]

            if (relative_sl is not None):
                # new stoploss relative to current_rate
                new_stoploss = (current_rate-relative_sl)/current_rate
                # turn into relative negative offset required by `custom_stoploss` return implementation
                result = new_stoploss - 1

        return result

```

Zie: https://www.freqtrade.io/en/stable/strategy-advanced/#custom-stoploss-using-an-indicator-from-dataframe-example

## Entry signalen verscherpen

timeframe: vooralsnog 2h
